In [ ]:
# lpr_scregseg environment

# Preprocess PBMC from a Healthy Donor - Granulocytes Removed Through Cell Sorting (10k) from 10x (10k_sorted)

Aim: Derive identical feature set for additional ATAC data from other studies as for treatment-control use case.

Keep cells with annotation from MOFA+ vignette (passing all respective preprocessing).

In [1]:
import os
import pandas as pd
from anndata import read_h5ad
import scanpy as sc
import scregseg
import matplotlib.pyplot as plt
import numpy as np

In [2]:
## ATAC preprocessing - derive same feature set as in treatment-control use case

In [3]:
wd = "liam_manuscript_reproducibility/data/original/10x/10k_sorted/"

In [4]:
! ls -l liam_manuscript_reproducibility/data/original/10x/10k_sorted/

total 2188768
-rw-r--r-- 1 prauten AG_Ohler 1988951230 Sep  6  2020 pbmc_granulocyte_sorted_10k_atac_fragments.tsv.gz
-rw-r--r-- 1 prauten AG_Ohler    1027204 Sep  6  2020 pbmc_granulocyte_sorted_10k_atac_fragments.tsv.gz.tbi
-rw-r--r-- 1 prauten AG_Ohler  162282142 Sep  6  2020 pbmc_granulocyte_sorted_10k_filtered_feature_bc_matrix.h5
-rw-r--r-- 1 prauten AG_Ohler   88984445 Sep  6  2020 pbmc_granulocyte_sorted_10k_per_barcode_metrics.csv


In [5]:
!scregseg fragments_to_counts \
          --fragmentfile liam_manuscript_reproducibility/data/original/10x/10k_sorted/pbmc_granulocyte_sorted_10k_atac_fragments.tsv.gz \
          --regions liam_manuscript_reproducibility/data/derived/Mimitou2021/ATAC/all_datasets_informative_rarest_states_out_of_50.bed \
          --with-fraglen \
          --counts liam_manuscript_reproducibility/data/derived/10x/10k_sorted/countmatrix_10k_sorted_informative_rarest_states_out_of_50_0.h5ad

***** WARNING: File /fast/AG_Ohler/prauten/projects/github/Liam_paper_reproducibility/data/original/10x/10k_sorted/pbmc_granulocyte_sorted_10k_atac_fragments.tsv.gz has inconsistent naming convention for record:
KI270728.1	134006	134070	ACTTGTCGTTGCGCGA-1	1

***** WARNING: File /fast/AG_Ohler/prauten/projects/github/Liam_paper_reproducibility/data/original/10x/10k_sorted/pbmc_granulocyte_sorted_10k_atac_fragments.tsv.gz has inconsistent naming convention for record:
KI270728.1	134006	134070	ACTTGTCGTTGCGCGA-1	1

***** WARNING: File /fast/AG_Ohler/prauten/projects/github/Liam_paper_reproducibility/data/original/10x/10k_sorted/pbmc_granulocyte_sorted_10k_atac_fragments.tsv.gz has inconsistent naming convention for record:
KI270728.1	134006	134070	ACTTGTCGTTGCGCGA-1	1

***** WARNING: File /fast/AG_Ohler/prauten/projects/github/Liam_paper_reproducibility/data/original/10x/10k_sorted/pbmc_granulocyte_sorted_10k_atac_fragments.tsv.gz has inconsistent naming convention for record:
KI270728.1	

In [6]:
# Load cell ranger meta data (10x pipeline)
cellranger_metadata = pd.read_csv(wd + 'pbmc_granulocyte_sorted_10k_per_barcode_metrics.csv')

In [7]:
# Keep only barcodes that were called as cells by 10x pipeline
cellranger_metadata = cellranger_metadata[cellranger_metadata.is_cell==1]

In [8]:
cellranger_metadata['gex_barcode'].to_csv('liam_manuscript_reproducibility/data/derived/10x/10k_sorted/' + 'qcontrolled_cells_10x_10k_sorted.csv', index=False)

In [9]:
print(f'{cellranger_metadata.shape[0]} high-quality cells are left for downstream processing')


11909 high-quality cells are left for downstream processing


In [11]:
# 128G
!scregseg subset \
          --incounts liam_manuscript_reproducibility/data/derived/10x/10k_sorted/countmatrix_10k_sorted_informative_rarest_states_out_of_50_0.h5ad \
          --outcounts liam_manuscript_reproducibility/data/derived/10x/10k_sorted/filtered_countmatrix_10k_sorted_informative_rarest_states_out_of_50_0.h5ad \
          --subset liam_manuscript_reproducibility/data/derived/10x/10k_sorted/qcontrolled_cells_10x_10k_sorted.csv